In [1]:
import pandas as pd
import numpy as np

# Load your edge_df (from previous step or csv)
edge_df = pd.read_csv('../../data/edges_for_gat.csv')

# Make unique identifiers for shifts (date+shift+ward)
edge_df['shift_id'] = edge_df['date'].astype(str) + "_" + edge_df['shift'] + "_" + edge_df['ward']

# Build mappings
nurse2idx = {n:i for i, n in enumerate(edge_df['nurse_id'].unique())}
shift2idx = {s:i+len(nurse2idx) for i, s in enumerate(edge_df['shift_id'].unique())}  # shift indices after nurses

# Add index columns
edge_df['nurse_idx'] = edge_df['nurse_id'].map(nurse2idx)
edge_df['shift_idx'] = edge_df['shift_id'].map(shift2idx)


In [2]:
import torch

edge_index = torch.tensor([
    edge_df['nurse_idx'].values,
    edge_df['shift_idx'].values
], dtype=torch.long)


C:\Users\Alex\AppData\Local\Temp\ipykernel_4120\1715278712.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  edge_index = torch.tensor([


In [3]:
num_nurses = len(nurse2idx)
num_shifts = len(shift2idx)
num_nodes = num_nurses + num_shifts

# Example: type encoding (nurse=0, shift=1)
x = torch.zeros((num_nodes, 1))
x[num_nurses:, 0] = 1


In [4]:
y = torch.tensor(edge_df['label'].values, dtype=torch.float)  # shape: [num_edges]


In [6]:
from torch_geometric.data import Data

data = Data(
    x=x,  # node features
    edge_index=edge_index,
    edge_attr=None,  # optional, can use None
    y=y
)


In [7]:
print(data)
print("Number of nodes:", data.num_nodes)
print("Number of edges:", data.num_edges)
print("Node features shape:", data.x.shape)


Data(x=[1859, 1], edge_index=[2, 3633], y=[3633])
Number of nodes: 1859
Number of edges: 3633
Node features shape: torch.Size([1859, 1])
